In [118]:
import os
from pathlib import Path
import geopandas as gpd
import xarray as xr
import pandas as pd
import numpy as np

mwb_flow_dir = r'C:\Users\CND905\Downloaded_Programs\mwb_flow'
os.chdir(mwb_flow_dir)

from mwb_flow.model_update import WB_Model
 

The model calls for:

dates as a vectors/1d array

temp and precip are ndarrays

and mon_Q is a pandas series with dates as index and flow in m^3/s


In [119]:
data_dict = xr.load_dataset(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\Examples\dataprep_Example1.1_PullData_output.nc')
data_dict

<xarray.Dataset> Size: 848B
Dimensions:       (location: 6, time: 4)
Coordinates:
  * location      (location) int64 48B 1000 2800 3000 5500 9500 9600
  * time          (time) datetime64[ns] 32B 2016-08-01 2016-09-01 ... 2016-11-01
    lat           (location) float64 48B 46.69 46.82 46.78 46.69 46.75 46.75
    lon           (location) float64 48B -114.5 -114.4 -114.4 ... -114.2 -114.2
    elev          (location) float64 48B 1.402e+03 1.435e+03 ... 1.335e+03
    area          (location) float64 48B 185.3 34.26 186.8 139.0 102.1 57.96
Data variables:
    mo_temp       (time, location) float64 192B 14.74 15.93 15.89 ... 2.77 2.919
    mo_precip     (time, location) float64 192B 2.66e+09 6.008e+08 ... 8.475e+09
    mo_discharge  (time, location) float64 192B 0.5594 0.04081 ... 2.86 3.099
Attributes:
    standard_name:  Monthly_Temperature
    units:          Celcius

In [128]:
data_dict.mo_temp.values

array([[14.73760997, 15.93294931, 15.89396655, 14.70680224, 16.27308468,
        16.55930521],
       [ 9.23348485, 10.04880952, 10.27141975,  8.92398551, 10.40052083,
        10.71064103],
       [ 4.00293255,  4.61313364,  4.84832736,  3.57791024,  4.91864919,
         5.22332506],
       [ 1.82030303,  2.56619048,  2.39555556,  1.72326087,  2.77020833,
         2.91897436]])

In [91]:
# # Precip and temp as ndarrays (1D)
# precip = data_dict.mo_precip.values[:,0]
# temp = data_dict.mo_temp.values[:,0]

# # latitude value
# lat = data_dict.coords["lat"].values[0]

# # discharge
# disch = pd.Series(data_dict.mo_discharge.values[:,0], index=data_dict.coords["time"].values)

# dates = data_dict.coords["time"].values


In [92]:
data_dict = xr.load_dataset(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\Examples\dataprep_Example1.1_PullData_output.nc')
soil_stor_cap = 73
init_per = 1
Temp_sno=0
Temp_rain=5
dr_frac=0.06
mlt_rate=0.6
rfactor=0.5

In [93]:
STC = soil_stor_cap
snotemp = Temp_sno
raintemp = Temp_rain
DRF = dr_frac
alpha = mlt_rate
rfactor = rfactor
latitude = data_dict.coords["lat"].values[0]
init_per = init_per
STo = 0.0
snw_st_init = 0.0
ST_inter = 0.0

# Calibration Parameter Ranges
snotemp_rng = [-10, 0]
raintemp_rng = [0, 10]
DRF_rng = [0.01, 0.99]
alpha_rng = [0.01, 0.99]
rfactor_rng = [0.01, 0.99]
# TODO edit code to accept xarray.datasets instead of a dictionary
# Updated latitude, T, P, dates, monthly_Q, days_in_mnth
# data_dict inputs are indexed to the first columns of arrays
T = data_dict.mo_precip.values[:,0]
P = data_dict.mo_precip.values[:,0]
dates = pd.to_datetime(data_dict.coords["time"].values)
monthly_Q = pd.Series(data_dict.mo_discharge.values[:,0], index=data_dict.coords["time"].values)
days_in_mnth = pd.to_datetime(data_dict.coords["time"].values).daysinmonth.to_numpy()

# gridded data for WB variables
snowfall = None
snow_stor = None
P_snow_melt = None
snowmelt = None
soil_m = None
PET = None
AET = None
rain = None
direct_runoff = None
surplus_runoff = None
runoff = None

In [94]:
# def _day_length(self):
rad_lat = np.radians(latitude)
days = dates - pd.Timedelta(15, unit='D')
J = days.dayofyear.values
decl = 0.409*np.sin((2*np.pi/365)*J - 1.39)
sunset_ang = np.arccos(-np.tan(rad_lat)*np.tan(decl))
Dl = (24/np.pi)*sunset_ang
Dl



array([15.19471356, 13.84293205, 12.18523799, 10.44712782])

# def _snow_input(self, Tsnow, Train)
Tsnow and Train are assigned from snotemp and raintemp in the run_model method. So we assign the Tsnow, and Train values now.

In [95]:
Tsnow = Temp_sno
Train = Temp_rain

# def _snow_input(self, Tsnow, Train):
## Snow Storage
## multidimensional array indices
sno_ind = np.where(T <= Tsnow)
mix_ind = np.where((T > Tsnow) & (T < Train))
rain_ind = np.where(T >= Train)
Psnow = P * 1.0
Psnow[sno_ind] = Psnow[sno_ind]
Psnow[mix_ind] = Psnow[mix_ind] * ((Train - T[mix_ind])/(Train - Tsnow))
Psnow[rain_ind] = 0.0
Psnow 

array([0., 0., 0., 0.])

# def _rain(self):

In [96]:
## Rain Fraction
# Prain = self.P - self.snowfall
Prain = P - Psnow
Prain

array([2.66048191e+09, 1.30239189e+10, 6.03137381e+10, 2.20693134e+10])

# def _direct_runoff(self, drf):

drf assigned from DRF here since it is not assinged until the run_model method

In [97]:
drf = DRF

# Dir_RO = drf * self.rain
Dir_RO = drf * Prain
Dir_RO

array([1.59628915e+08, 7.81435133e+08, 3.61882429e+09, 1.32415880e+09])

# def _snowmelt(self, alpha, Tsnow):
Unsure of why there are 3 index for the days_in_month array so hashed the line till understood.

In [98]:

# Sno_M = alpha * (T - Tsnow) * days_in_mnth[:,None,None]
Sno_M = alpha * (T - Tsnow) * days_in_mnth
Sno_M[Sno_M < 0.0] = 0.0
Sno_M

array([4.94849635e+10, 2.34430540e+11, 1.12183553e+12, 3.97247641e+11])

# def _PET(self):
Also hashed the line using 3 indexes here

In [99]:
e_sat = 6.108 * np.exp((17.27 * T) / (T + 237.3))
rho_sat = 216.7 * (e_sat / (T + 273.3))

# D = self._day_length()
D = Dl

Kpec = 1.0
# PET = 0.1651 * self.days_in_mnth[:,None, None] * (D[:,None,None]/12) * rho_sat * Kpec
PET = 0.1651 * days_in_mnth * (D/12) * rho_sat * Kpec

PET

array([102.01937453,  18.3737153 ,   3.60885059,   8.18311633])

# def _soilmoisture_storage(self, dates, time_offset, Tsnow, Train, drf, alpha, rfactor):

warmup_dates, warmup_off and time_offset are defined here since they are later defined in the calibrate method

In [116]:
# Change so we just get an index range for warm up and calibration
init_per = 2

warmup_dates = dates[0:init_per]
warmup_off = np.where(dates == warmup_dates[-1])[0][0]
time_offset=warmup_off
time_offset


np.int64(1)

In [ ]:
warmup_dates = self.dates[0:self.init_per]
warmup_off = np.where(dts == warmup_dates[0])[0][0]
time_offset=warmup_off

# self.snowfall = self._snow_input(Tsnow, Train) 
snowfall = Psnow

# self.rain = self._rain()
rain = Prain


# self.direct_runoff = self._direct_runoff(drf)
direct_runoff = Dir_RO


# self.P_snow_melt = self._snowmelt(alpha, Tsnow)
P_snow_melt = Sno_M


# self.PET = self._PET()
PET = PET


smstor = []
sm_in = []
AET_l = []
surp_r = []
snowstor = []
snwmlt = []
for i in tqdm(np.arange(time_offset, len(dates))):
    
    if i == 0:
        STi_1 = self.STo
        sno_stori_1 = self.snw_st_init
        STmi_1 = self.ST_inter
    else:
        STi_1 = smstor[i-1]
        sno_stori_1 = snowstor[i-1]
        STmi_1 = sm_in[i-1]
    
    ## Liquid water input to Soil Moisture Storage
    snow_storage = sno_stori_1 + snowfall[i,:,:] - P_snow_melt[i,:,:]
    
    hi_snwm = np.where(snow_storage > 0.0)
    lo_snwm = np.where(snow_storage <= 0.0)
    
    # snwm = self.P_snow_melt[i,:,:] * 1.0
    snwm = P_snow_melt[i,:,:] * 1.0

    snwm[hi_snwm] = P_snow_melt[i,:,:][hi_snwm]
    if type(sno_stori_1) == float:
        snwm[lo_snwm] = sno_stori_1 + snowfall[i,:,:][lo_snwm]
    else:
        snwm[lo_snwm] = sno_stori_1[lo_snwm] + self.snowfall[i,:,:][lo_snwm]

    snow_storage[snow_storage < 0.0] = 0.0
    
    P_tot = (rain[i,:,:] - direct_runoff[i,:,:]) + snwm

    ## multidimensional array indices
    hi_pet = np.where(P_tot >= self.PET[i,:,:])
    lo_pet = np.where(P_tot < self.PET[i,:,:])
    
    AET = self.PET[i,:,:] * 1.0
    AET[hi_pet] = AET[hi_pet]
    if type(STi_1) == float:
        AET[lo_pet] = P_tot[lo_pet] + (abs(P_tot[lo_pet] - self.PET[i,:,:][lo_pet]) * (STi_1/self.STC))
    else:
        AET[lo_pet] = P_tot[lo_pet] + (abs(P_tot[lo_pet] - self.PET[i,:,:][lo_pet]) * (STi_1[lo_pet]/self.STC))
    STi = STi_1 + P_tot - AET
    STmi = STi * 1.0
    sm_in.append(STmi)
    STi[STi < 0] = 0.0
    STi[STi > self.STC] = self.STC
    ## Surplus Surface Runoff
    SurRun_o = STmi_1 - self.STC
    if type(STmi_1) != float:
        SurRun_o[SurRun_o < 0] = 0.0
    else:
        SurRun_o = 0.0
    SurRun_i = STmi - self.STC
    SurRun_i[SurRun_i < 0] = 0.0
    SR = (SurRun_i * rfactor) + (SurRun_o * (1-rfactor))
    surp_r.append(SR)
    smstor.append(STi)
    AET_l.append(AET)
    snowstor.append(snow_storage)
    snwmlt.append(snwm)
ST = np.ma.array(smstor, mask=self.P[time_offset:len(dates),:,:].mask)
ST_Int = np.ma.array(sm_in, mask=self.P[time_offset:len(dates),:,:].mask)
AET = np.ma.array(AET_l, mask=self.P[time_offset:len(dates),:,:].mask)
SR = np.ma.array(surp_r, mask=self.P[time_offset:len(dates),:,:].mask)
SnwStor = np.ma.array(snowstor, mask=self.P[time_offset:len(dates),:,:].mask)
snowmlt = np.ma.array(snwmlt, mask=self.P[time_offset:len(dates),:,:].mask)

return dict(SoilStor=ST, Interm_SoilStor=ST_Int, AET=AET, SurplusRunoff=SR, SnowStorage=SnwStor, Snowmelt=snowmlt)

# def_run_model(self, Tsnow=None, Train=None, drf=None, alpha=None, rfactor=None):

In [ ]:
    def run_model(self, Tsnow=None, Train=None, drf=None, alpha=None, rfactor=None):
        kwargs = dict(Tsnow=Tsnow,
                      Train=Train,
                      drf=drf,
                      alpha=alpha,
                      rfactor=rfactor)
        
        selfkwargs = dict(Tsnow=snotemp,
                          Train=raintemp,
                          drf=DRF,
                          alpha=alpha,
                          rfactor=rfactor)
        
        for key in kwargs:
            if kwargs[key] == None:
                kwargs[key] = selfkwargs[key]
                 
        warmup_dates = dates[0:init_per]
        warmup_off = np.where(dates == warmup_dates[0])[0][0]

        w_args = [warmup_dates, warmup_off, kwargs['Tsnow'], kwargs['Train'], kwargs['drf'], kwargs['alpha'], kwargs['rfactor']]
        # w_args = dict(dates=warmup_dates, time_offset=warmup_off)

        args = [dates, 0, kwargs['Tsnow'], kwargs['Train'], kwargs['drf'], kwargs['alpha'], kwargs['rfactor']]
        
        # wf_kwargs = w_args.update(kwargs)
        # f_kwargs = args.update(kwargs)
        
        # model warm up
        print("Initializing Model")
        self.STo, self.snw_st_init, self.ST_inter = self._model_warmup(*w_args)
        # model run
        print("Running Model")
        Reslt = self._soilmoisture_storage(*args)
        # assign final arrays
        AET = Reslt['AET']
        soil_m = Reslt['SoilStor']
        surplus_runoff = Reslt['SurplusRunoff']
        snow_stor = Reslt['SnowStorage']
        snowmelt = Reslt['Snowmelt']
        # compute total runoff
        Tot_Runoff = surplus_runoff + direct_runoff
        runoff = Tot_Runoff
        # convert to streamflow in cms
        Qarrays = ((Tot_Runoff/1000) * 30**2) / (86400 * days_in_mnth.values[:,None, None])
        Qvec = Qarrays.sum(axis=(1,2))
        Qs = pd.Series(Qvec, index=dates)
        return Qs